In [1]:
import pandas as pd
import re
import io

print("Libraries imported successfully!")

Libraries imported successfully!


In [3]:
# Tentukan path ke file Anda
file_path = '../data/raw/news/news_with_images.csv'

try:
    df = pd.read_csv(file_path)
    print(f"File '{file_path}' berhasil dimuat.")
    print(f"Bentuk data (baris, kolom): {df.shape}")
except FileNotFoundError:
    print(f"Error: File '{file_path}' tidak ditemukan.")
except Exception as e:
    print(f"Error saat memuat file: {e}")

File '../data/raw/news/news_with_images.csv' berhasil dimuat.
Bentuk data (baris, kolom): (1999, 20)


In [4]:
# Tampilkan info() untuk melihat tipe data dan data hilang
print("--- Tinjauan Data Awal (df.info()) ---")
buffer = io.StringIO()
df.info(buf=buffer)
info_str = buffer.getvalue()
print(info_str)

--- Tinjauan Data Awal (df.info()) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999 entries, 0 to 1998
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1999 non-null   int64 
 1   title           1999 non-null   object
 2   url             1999 non-null   object
 3   domain          1999 non-null   object
 4   source          1999 non-null   object
 5   topic           1999 non-null   object
 6   date_utc        1999 non-null   object
 7   authors         1999 non-null   object
 8   modality        1999 non-null   object
 9   language        1999 non-null   object
 10  text_len        1999 non-null   int64 
 11  text_preview    1999 non-null   object
 12  image_url       1999 non-null   object
 13  image_path      1984 non-null   object
 14  label           1999 non-null   int64 
 15  created_at_utc  1999 non-null   object
 16  notes           1999 non-null   object
 17  text         

In [5]:
# Tampilkan 5 baris pertama
print("--- Tinjauan Data Awal (df.head()) ---")
pd.set_option('display.max_columns', None) # Tampilkan semua kolom
pd.set_option('display.max_colwidth', 100) # Batasi lebar kolom agar rapi
print(df.head())

--- Tinjauan Data Awal (df.head()) ---
   id                                                                  title  \
0   1  Profil Menko Hukum HAM Yusril Ihza Mahendra di Kabinet Prabowo-Gibran   
1   2    Profil Menteri PPPA Arifatul Choiri Fauzi di Kabinet Prabowo-Gibran   
2   3                  Prabowo Tunjuk Yassierli Jadi Menteri Ketenagakerjaan   
3   4       Profil Yassierli, Menteri Ketenagakerjaan Kabinet Prabowo-Gibran   
4   5    Apa yang Harus Dilakukan di Usia 30 Tahun untuk Hidup Panjang Umur?   

                                                                                                   url  \
0  https://nasional.kompas.com/read/2024/10/20/22533711/profil-menko-hukum-ham-yusril-ihza-mahendra...   
1  https://nasional.kompas.com/read/2024/10/20/23214911/profil-menteri-pppa-arifatul-choiri-fauzi-d...   
2  https://nasional.kompas.com/read/2024/10/20/22470301/prabowo-tunjuk-yassierlijadi-menteri-ketena...   
3  https://www.kompas.com/tren/read/2024/10/20/224623365

In [6]:
print(f"Jumlah baris sebelum dihapus: {len(df)}")
print(f"Jumlah data 'text' yang hilang: {df['text'].isnull().sum()}")

# Buat DataFrame baru (best practice) atau modifikasi langsung
df_cleaned = df.dropna(subset=['text']).copy()

print(f"Jumlah baris setelah dihapus: {len(df_cleaned)}")
print(f"Data 'text' yang hilang sekarang: {df_cleaned['text'].isnull().sum()}")

Jumlah baris sebelum dihapus: 1999
Jumlah data 'text' yang hilang: 38
Jumlah baris setelah dihapus: 1961
Data 'text' yang hilang sekarang: 0


In [7]:
columns_to_drop = ['period', 'source_csv']
existing_cols_to_drop = [col for col in columns_to_drop if col in df_cleaned.columns]

if existing_cols_to_drop:
    df_cleaned = df_cleaned.drop(columns=existing_cols_to_drop)
    print(f"Kolom berikut berhasil dihapus: {', '.join(existing_cols_to_drop)}")
else:
    print("Kolom 'period' dan 'source_csv' tidak ditemukan (mungkin sudah dihapus).")

Kolom berikut berhasil dihapus: period, source_csv


In [8]:
try:
    df_cleaned['date_utc'] = pd.to_datetime(df_cleaned['date_utc'])
    df_cleaned['created_at_utc'] = pd.to_datetime(df_cleaned['created_at_utc'])
    print("Kolom 'date_utc' dan 'created_at_utc' berhasil dikonversi ke datetime.")
    print("\nTipe data baru:")
    print(df_cleaned[['date_utc', 'created_at_utc']].dtypes)
except Exception as e:
    print(f"Gagal mengonversi kolom tanggal: {e}")

Kolom 'date_utc' dan 'created_at_utc' berhasil dikonversi ke datetime.

Tipe data baru:
date_utc          datetime64[ns, UTC]
created_at_utc    datetime64[ns, UTC]
dtype: object


In [9]:
def basic_text_clean(text):
    """
    Fungsi untuk membersihkan teks dari karakter newline dan spasi berlebih.
    """
    if isinstance(text, str):
        # 1. Ganti karakter newline, tab, atau return dengan spasi
        text = re.sub(r'[\n\r\t]+', ' ', text)
        # 2. Ganti spasi ganda atau lebih dengan satu spasi
        text = re.sub(r'\s+', ' ', text)
        # 3. Hapus spasi di awal dan akhir
        return text.strip()
    return text

print("Fungsi 'basic_text_clean' berhasil dibuat.")

Fungsi 'basic_text_clean' berhasil dibuat.


In [10]:
print("Membersihkan kolom 'text' dan 'title'...")

df_cleaned['text_cleaned'] = df_cleaned['text'].apply(basic_text_clean)
df_cleaned['title_cleaned'] = df_cleaned['title'].apply(basic_text_clean)

print("Kolom baru 'text_cleaned' dan 'title_cleaned' berhasil dibuat.")

Membersihkan kolom 'text' dan 'title'...
Kolom baru 'text_cleaned' dan 'title_cleaned' berhasil dibuat.


In [12]:
print(f"Jumlah baris sebelum 'image_path' kosong dihapus: {len(df_cleaned)}")
print(f"Jumlah data 'image_path' yang hilang: {df_cleaned['image_path'].isnull().sum()}")

# Menghapus baris di mana 'image_path' adalah NaN
df_cleaned = df_cleaned.dropna(subset=['image_path'])

print(f"Jumlah baris setelah 'image_path' kosong dihapus: {len(df_cleaned)}")
print(f"Data 'image_path' yang hilang sekarang: {df_cleaned['image_path'].isnull().sum()}")

Jumlah baris sebelum 'image_path' kosong dihapus: 1961
Jumlah data 'image_path' yang hilang: 15
Jumlah baris setelah 'image_path' kosong dihapus: 1946
Data 'image_path' yang hilang sekarang: 0


In [13]:
print("--- Tinjauan Data Setelah Pembersihan (df_cleaned.info()) ---")
buffer = io.StringIO()
df_cleaned.info(buf=buffer)
info_str = buffer.getvalue()
print(info_str)

--- Tinjauan Data Setelah Pembersihan (df_cleaned.info()) ---
<class 'pandas.core.frame.DataFrame'>
Index: 1946 entries, 0 to 1998
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   id              1946 non-null   int64              
 1   title           1946 non-null   object             
 2   url             1946 non-null   object             
 3   domain          1946 non-null   object             
 4   source          1946 non-null   object             
 5   topic           1946 non-null   object             
 6   date_utc        1946 non-null   datetime64[ns, UTC]
 7   authors         1946 non-null   object             
 8   modality        1946 non-null   object             
 9   language        1946 non-null   object             
 10  text_len        1946 non-null   int64              
 11  text_preview    1946 non-null   object             
 12  image_url       1946 non-null   o

In [14]:
print("--- Tinjauan Data Setelah Pembersihan (df_cleaned.head()) ---")

# Tampilkan kolom-kolom kunci yang sudah dimodifikasi
print(df_cleaned[['title_cleaned', 'text_cleaned', 'date_utc', 'image_path', 'label']].head())

--- Tinjauan Data Setelah Pembersihan (df_cleaned.head()) ---
                                                           title_cleaned  \
0  Profil Menko Hukum HAM Yusril Ihza Mahendra di Kabinet Prabowo-Gibran   
1    Profil Menteri PPPA Arifatul Choiri Fauzi di Kabinet Prabowo-Gibran   
2                  Prabowo Tunjuk Yassierli Jadi Menteri Ketenagakerjaan   
3       Profil Yassierli, Menteri Ketenagakerjaan Kabinet Prabowo-Gibran   
4    Apa yang Harus Dilakukan di Usia 30 Tahun untuk Hidup Panjang Umur?   

                                                                                          text_cleaned  \
0  JAKARTA, KOMPAS.com - Profil Yusril Ihza Mahendra kembali menjadi sorotan publik setelah ditunju...   
1  JAKARTA, KOMPAS.com - Profil Arifatul Choiri Fauzi menjadi sorotan setelah ditunjuk menjadi Ment...   
2  JAKARTA, KOMPAS.com - Presiden Prabowo Subianto menunjuk Guru Besar Institut Teknologi Bandung (...   
3  Kompas.com, 20 Oktober 2024, 22:46 WIB Erwina Rachmi P

In [15]:
cleaned_file_path = 'AllMetadata_Cleaned_v3.csv'

try:
    df_cleaned.to_csv(cleaned_file_path, index=False)
    print(f"\nProses pembersihan selesai!")
    print(f"Data bersih telah disimpan ke: {cleaned_file_path}")
except Exception as e:
    print(f"Error saat menyimpan file: {e}")


Proses pembersihan selesai!
Data bersih telah disimpan ke: AllMetadata_Cleaned_v3.csv
